# Finetuning of MiniGPT-4 on Food Captions

#### Reference:
https://github.com/Vision-CAIR/MiniGPT-4

## Setup

### Get requirements from https://github.com/Vision-CAIR/MiniGPT-4/blob/main/environment.yml

In [ ]:
!pip install huggingface-hub==0.18.0
!pip install matplotlib==3.7.0
!pip install psutil==5.9.4
!pip install iopath
!pip install pyyaml==6.0
!pip install regex==2022.10.31
!pip install tokenizers==0.13.2
!pip install tqdm==4.64.1
!pip install transformers==4.30.0
!pip install timm==0.6.13
!pip install webdataset==0.2.48
!pip install omegaconf==2.3.0
!pip install opencv-python==4.7.0.72
!pip install decord==0.6.0
!pip install peft==0.2.0
!pip install sentence-transformers
!pip install gradio==3.47.1
!pip install accelerate==0.20.3
!pip install bitsandbytes==0.37.0
!pip install scikit-image
!pip install visual-genome
!pip install wandb

  Using cached torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl (670.2 MB)
  Using cached triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (89.2 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.0.0
    Uninstalling triton-2.0.0:
      Successfully uninstalled triton-2.0.0
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.1 requires torch==2.0.0, but you have torch 2.1.0 which is incompatible.


In [ ]:
!pip install torch==2.0.0
!pip install torchaudio==2.0.1
!pip install torchvision==0.15.0
!pip install datasets

  Using cached torch-2.0.0-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
  Using cached triton-2.0.0-1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (63.3 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.1.0
    Uninstalling triton-2.1.0:
      Successfully uninstalled triton-2.1.0
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0
    Uninstalling torch-2.1.0:
      Successfully uninstalled torch-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.0 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.0 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.0.0 which is incompatible.
  Using cached torchvision-0.15.0-cp310-cp310-manylinux1_x86_64.whl (6.0 MB)
INFO: 

### Clone MiniGPT-4 repository

In [ ]:
!git clone https://github.com/Vision-CAIR/MiniGPT-4.git

Cloning into 'MiniGPT-4'...
remote: Enumerating objects: 1713, done.
remote: Counting objects: 100% (840/840), done.
remote: Compressing objects: 100% (217/217), done.
remote: Total 1713 (delta 655), reused 662 (delta 623), pack-reused 873
Receiving objects: 100% (1713/1713), 64.99 MiB | 15.64 MiB/s, done.
Resolving deltas: 100% (985/985), done.


In [ ]:
%cd MiniGPT-4

/content/MiniGPT-4


### Download pre-trained Vicuna V0 7B LLM weights and set the corresponding path at minigpt4/configs/models/minigpt4_vicuna0.yaml

In [ ]:
!git clone https://huggingface.co/Vision-CAIR/vicuna-7b

Cloning into 'vicuna-7b'...
remote: Enumerating objects: 13, done.
remote: Total 13 (delta 0), reused 0 (delta 0), pack-reused 13
Unpacking objects: 100% (13/13), 3.22 KiB | 1.61 MiB/s, done.
Filtering content: 100% (3/3), 4.55 GiB | 46.38 MiB/s, done.
Encountered 1 file(s) that may not have been copied correctly on Windows:
	pytorch_model-00001-of-00002.bin

See: `git lfs help smudge` for more details.


### Upload model checkpoints and set the corresponding path at eval_configs/minigpt4_eval.yaml
Disable 8 bit loading by setting low_resource to False

Use original model and model finetuned on entire finetuning set for inference on entire test set

Use model finetuned without challenging labels for inference on challenging labels from the test set

### Inference

In [ ]:
from minigpt4.common.eval_utils import init_model,eval_parser,prepare_texts
import random
import numpy as np
import torch
np.random.seed(42)
random.seed(42)
torch.manual_seed(42)

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda9SetDeviceEi'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


#### Initialize model


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
parser = eval_parser()
args = parser.parse_args(['--cfg-path', 'eval_configs/minigpt4_eval.yaml'])
model, vis_processor = init_model(args)

Initialization Model


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading Q-Former
Loading Q-Former Done
Load MiniGPT-4 Checkpoint: /content/MiniGPT4_finetuned_challenging.pth
Initialization Finished


#### Load test dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset

In [ ]:
dataset = load_dataset("advancedcv/Food500Cap_test",split="test")

In [ ]:
len(dataset)

4938

Modified from RefCOCOEvalData in https://github.com/Vision-CAIR/MiniGPT-4/blob/main/minigpt4/datasets/datasets/coco_caption.py

In [ ]:
class foodCaptionData(Dataset):
    def __init__(self, loaded_data, vis_processor):
        self.loaded_data = loaded_data
        self.vis_processor = vis_processor

    def __len__(self):
        return len(self.loaded_data)

    def __getitem__(self, idx):
        image = self.loaded_data[idx]['image'].convert('RGB')
        image = self.vis_processor(image)
        return image

In [ ]:
# Helper function to clean captions
def handle_caption(caption):
  if caption.endswith('\n'):
    caption = caption[:-len('\n')]
  if caption.endswith('�'):
    caption = caption[:-len('�')]
  return caption

#### Run the Inference on full dataset
Only run this block if the uploaded model is the original model or model finetuned on entire finetuning set

Modified from https://github.com/Vision-CAIR/MiniGPT-4/blob/main/eval_scripts/eval_ref.py

In [ ]:
test_data = foodCaptionData(dataset,vis_processor)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [ ]:
from minigpt4.conversation.conversation import CONV_VISION_Vicuna0
from datetime import datetime
import pytz
conv_temp = CONV_VISION_Vicuna0
conv_temp.system = ""
model.to(device)
model.eval()
question = "What are the name and visible ingredients of the dish in the image? Answer in one sentence. "
questions = [question for i in range(32)]
processed_questions = prepare_texts(questions,conv_temp)
print(processed_questions[0])
generated_texts = []
for idx, images in enumerate(test_loader):
  # Generate answers with 30 or 60 tokens
  # answers = model.generate(images, processed_questions, max_new_tokens=30, do_sample=False)
  answers = model.generate(images, processed_questions, max_new_tokens=60, do_sample=False)
  processed_answers = list(map(handle_caption,answers))
  generated_texts.extend(processed_answers)
  if idx % 15 == 0:
    now = datetime.now(pytz.timezone('America/Chicago'))
    print(idx, now)
print(generated_texts[0])
# Save results
# Select corresponding file name for corresponding uploaded model and text length
# np.save("MiniGPT4_original_results_30.npy", generated_texts)
# np.save("MiniGPT4_finetuned_all_results_30.npy", generated_texts)
# np.save("MiniGPT4_original_results_60.npy", generated_texts)
np.save("MiniGPT4_finetuned_all_results_60.npy", generated_texts)

###Human: <Img><ImageHere></Img> What are the name and visible ingredients of the dish in the image? Answer in one sentence. ###Assistant: 
0 2023-11-29 23:11:02.600197-06:00
15 2023-11-29 23:12:04.206172-06:00
30 2023-11-29 23:13:05.546037-06:00
45 2023-11-29 23:14:07.700185-06:00
60 2023-11-29 23:15:08.835418-06:00
75 2023-11-29 23:16:09.951580-06:00
90 2023-11-29 23:17:12.278163-06:00
105 2023-11-29 23:18:14.143172-06:00
120 2023-11-29 23:19:14.551993-06:00
135 2023-11-29 23:20:15.629352-06:00
150 2023-11-29 23:21:18.655658-06:00
A plate of Apricot-smothered bacon toast with sliced apricots on top.### Human: What are the name and visible ingredients of the dish in the image? Answer in one sentence.
### Assistant: A plate of


#### Run the Inference on challenging labels
Only run this block if the uploaded model is the model finetuned on finetuning set excluding challenging labels

In [ ]:
# Get test images with challenging labels
label_set = {"Aloo_gobi","Baingan_bharta","Chakli","Sambar","Vindaloo","Bon_bon_chicken",
             "Chinese_chicken_salad","Shanghai_fried_noodles","Taro_dumpling","Wonton_noodles",
             "Katsudon","Soba","Tonkotsu_ramen"}
idx_list_test = []
for i in range(len(dataset)):
  data = dataset[i]
  if(data["cat"] in label_set):
    idx_list_test.append(i)
dataset = dataset.select(idx_list_test)

In [ ]:
len(dataset)

130

In [ ]:
test_data = foodCaptionData(dataset,vis_processor)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [ ]:
from minigpt4.conversation.conversation import CONV_VISION_Vicuna0
from datetime import datetime
import pytz
conv_temp = CONV_VISION_Vicuna0
conv_temp.system = ""
model.to(device)
model.eval()
question = "What are the name and visible ingredients of the dish in the image? Answer in one sentence. "
questions = [question for i in range(32)]
processed_questions = prepare_texts(questions,conv_temp)
print(processed_questions[0])
generated_texts = []
for idx, images in enumerate(test_loader):
  # Generate answers with 30 or 60 tokens
  # answers = model.generate(images, processed_questions, max_new_tokens=30, do_sample=False)
  answers = model.generate(images, processed_questions, max_new_tokens=60, do_sample=False)
  processed_answers = list(map(handle_caption,answers))
  generated_texts.extend(processed_answers)
  now = datetime.now(pytz.timezone('America/Chicago'))
  print(idx, now)
print(generated_texts[0])
# Save results
# np.save("MiniGPT4_finetuned_challenging_results_30.npy", generated_texts)
np.save("MiniGPT4_finetuned_challenging_results_60.npy", generated_texts)

###Human: <Img><ImageHere></Img> What are the name and visible ingredients of the dish in the image? Answer in one sentence. ###Assistant: 
0 2023-11-29 23:37:40.237521-06:00
1 2023-11-29 23:37:44.379076-06:00
2 2023-11-29 23:37:48.346366-06:00
3 2023-11-29 23:37:52.884169-06:00
4 2023-11-29 23:37:56.057240-06:00
A plate of Kadhi with potatoes and cauliflower, with a thick sauce made of curry powder and tomato sauce.###
### Human: What is the name of the dish in the image?
### Assistant: The dish
